## Plotando dados de arquivos NetCDF em Python
- Fonte: Robson Passos, para acessar arquivo original [clique aqui](https://gist.github.com/Robson-Passos/7696f490fa14a7bedba118dddd10a628)
- Dado utilizado: [ERA5 hourly data on pressure levels from 1940 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-pressure-levels?tab=overview)
- As variáveis dos dados ERA5 de reanalize foram: componente zonal do vento (leste-oeste); componente meridional do vento (norte-sul); e temperatura; níveis de pressão (250 hPa, 500 hPa, 850 hPa e 1000 hPa); dias 29, 30 e 31 do mês de dezembro do ano de 2019; horários: 00:00, 06:00, 12:00 e 18:00.
- Link para baixar o dado utilizado: [Acesse aqui](https://drive.google.com/drive/folders/1jlDHxLQoYFTxwWb4kYCIFmRMTn8xR_jl?usp=sharing)

### Instalando bibliotecas/pacotes

In [ ]:
!pip install cartopy -q

### Importando bibliotecas

In [ ]:
#Importando bibliotecas/pacotes
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

###Abrindo o arquivo netCDF

In [ ]:
#Abrindo arquivo NetCDF
ds = xr.open_dataset('/content/drive/MyDrive/00_Disc_Meteo_Pratica_Python/uvt_2019.nc')
ds

### Extraindo dados para a área da América do Sul

In [ ]:
ds_subset = ds.sel(longitude=slice(-90, -20), latitude=slice(10, -60))
ds_subset

In [ ]:
#Salvando o dataset como um novo arquivo NetCDF
ds_subset.to_netcdf('/content/drive/MyDrive/00_Disc_Meteo_Pratica_Python/ds_AS.nc')

###Extraindo as variáveis

In [ ]:
u = ds_subset['u']
v = ds_subset['v']
t = ds_subset['t'] - 273.15 #convertendo de Kelvin para Celsus

wspd = (u**2 + v**2)**(0.5) #Calculando a velocidade do vento

In [ ]:
u

###Indexando e selecionando dados

In [ ]:
t.valid_time

In [ ]:
#Selecionando pelo índice
t.isel(valid_time=slice(0, 5))

In [ ]:
#Selecionando pelos labels
t.sel(valid_time=slice('2019-12-29T00:00:00', '2019-12-30T00:00:00'))

###Extrair dados de um ponto de grade

In [ ]:
t.pressure_level

In [ ]:
t.sel(longitude=-45.2, latitude=-22.3, method='nearest').isel(pressure_level=1).plot()

###Extraindo um perfil vertical

In [ ]:
wspd.isel(valid_time=0).sel(longitude=-45.2, latitude=-22.3, method='nearest').plot(y='pressure_level',
                                                                              yincrease=False)

###Gerando mapa

In [ ]:
t.pressure_level

In [ ]:
#Gerando um mapa da temperatura para o tempo '2019-12-31T18:00:00' e o nível de 1000 hPa
# t.sel(valid_time='2019-12-31T18:00:00').isel(pressure_level=3).plot(cmap='plasma')
t.sel(valid_time='2019-12-31T18:00:00').isel(pressure_level=0).plot(cmap='viridis')

In [ ]:
#Gerando um mapa com a temperatura média
t.sel(pressure_level=1000).mean(dim='valid_time').plot()

###Gerando um mapa global com os dados de temperatura

In [ ]:
u = ds['u']
v = ds['v']
t = ds['t'] - 273.15 #convertendo de Kelvin para Celsus

wspd = (u**2 + v**2)**(0.5) #Calculando a velocidade do vento

In [ ]:
#Criando um objeto de figura para receber o mapa
fig = plt.figure(figsize=(10, 8))

#Criando um eixo com uma determinada projeção
ax = fig.add_subplot(111, projection=ccrs.PlateCarree()) #pode ver outras projeções na documentação do pacote

#Criando matrizes de longitude e latitude
lon, lat = np.meshgrid(t.longitude, t.latitude)

#Plotando a temperatura do ar para o globo todo
im = ax.contourf(lon, lat, t.isel(valid_time=0, pressure_level=0),
                 levels=np.arange(-45, 46, 2), cmap='jet',
                 transform=ccrs.PlateCarree())

#Inserindo uma colorbar
cbar = plt.colorbar(im, ax=ax, pad=0.06, fraction=0.023)
cbar.set_label(label='Temperatura (°C)', size=20)
cbar.ax.tick_params(labelsize=12)

#Adicionando o contorno dos continentes
ax.add_feature(cfeature.COASTLINE)

#Adicionando os limites dos países
ax.add_feature(cfeature.BORDERS)

#Adicionando os limites estaduais
# Updated: Using 10m scale instead of 50m
states = cfeature.NaturalEarthFeature(category='cultural',
                                      name='admin_1_states_provinces_lines',
                                      scale='10m',
                                      facecolor='none')
ax.add_feature(states, edgecolor='k', linestyle='--')


#Adicionando o título da figura
ax.set_title('Temperatura do ar (°C) 2019-12-29 00Z', fontsize=20)

#Adicionando as linhas de grade
g1 = ax.gridlines(crs=ccrs.PlateCarree(), linestyle='--', color='gray', draw_labels=True)

#Removendo os labels do topo e da direita
g1.top_labels = False
g1.right_labels = False

#Formatando os labels como latitude e longitude
g1.yformatter = LATITUDE_FORMATTER
g1.xformatter = LONGITUDE_FORMATTER

g1.xlabel_style = {'size': 10}
g1.ylabel_style = {'size': 10}

#Salvando o mapa como uma figura
plt.savefig('Mapa_temp_global.png', dpi=300, bbox_inches='tight')

# plt.savefig('/content/drive/MyDrive/00_Disc_Meteo_Pratica_Python/Mapa_temp_global_v2.png', dpi=300, bbox_inches='tight')

###Gerando um mapa da AS com as linhas de corrente

In [ ]:
#Extraindo um subset da América do Sul
u_as = u.sel(longitude=slice(-90, -20), latitude=slice(10, -60))
v_as = v.sel(longitude=slice(-90, -20), latitude=slice(10, -60))
wspd_as = wspd.sel(longitude=slice(-90, -20), latitude=slice(10, -60))

In [ ]:
#Criando um objeto de figura para receber o mapa
fig = plt.figure(figsize=(8, 6))

#Criando um eixo com uma determinada projeção
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

#Definindo a extensão do mapa
ax.set_extent([-90, -20, 10, -60])

#Criando matrizes de longitude e latitude
lon, lat = np.meshgrid(u_as.longitude, u_as.latitude)

#Plotando as linhas de corrente e a intensidade do vento
strm = ax.streamplot(lon, lat, u_as.isel(valid_time=0, pressure_level=0).values, v_as.isel(valid_time=0, pressure_level=3).values,
                     color=wspd_as.isel(valid_time=0, pressure_level=0).values, cmap='jet',
                     transform=ccrs.PlateCarree())

#Inserindo uma colorbar
cbar = fig.colorbar(strm.lines)
cbar.set_label(label='Vel. do vento (ms$^{-1}$)', size=20, weight='normal')
cbar.ax.tick_params(labelsize=12)

#Adicionando o contorno dos continentes
ax.add_feature(cfeature.COASTLINE)

#Adicionando os limites dos países
ax.add_feature(cfeature.BORDERS)

#Adicionando os limites estaduais
states = cfeature.NaturalEarthFeature(category='cultural',
                                      name='admin_1_states_provinces_lines', # Changed name to 'admin_1_states_provinces_lines'
                                      scale='10m',
                                      facecolor='none')
ax.add_feature(states, edgecolor='k', linestyle='--')

#Adicionando o título da figura
ax.set_title('Linhas de corrente (°C) - 2019-12-29 00Z', fontsize=14, y=1.02)

#Adicionando as linhas de grade
g1 = ax.gridlines(crs=ccrs.PlateCarree(), linestyle='--', color='gray', draw_labels=True)

#Removendo os labels do topo e da direita
g1.top_labels = False
g1.right_labels = False

#Formatando os labels como latitude e longitude
g1.yformatter = LATITUDE_FORMATTER
g1.xformatter = LONGITUDE_FORMATTER

#Mofificando o tamanho da fonte dos labels
g1.xlabel_style = {'size': 14}
g1.ylabel_style = {'size': 14}

#Salvando o mapa como uma figura
plt.savefig('Mapa_streamlines_AS.png', dpi=300, bbox_inches='tight')
plt.savefig('/content/drive/MyDrive/00_Disc_Meteo_Pratica_Python/Mapa_streamlines_AS.png', dpi=300, bbox_inches='tight')
#